# Fundamentos de estadística para Analítica de Datos





In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats # libreria estadistica de Scipy
from sklearn.feature_selection import RFE # RFE es para seleccionar modelos
from sklearn.model_selection import train_test_split # para dividir la base en train y test 
from sklearn import linear_model # para modelo lineal
from sklearn.metrics import mean_squared_error, r2_score # para sacar las metricas 
import statsmodels.api as sm  ## Parte estadistica
from statsmodels.sandbox.regression.predstd import wls_prediction_std  ## Parte estadistica

# Regresión lineal
En una regresión se tiene una variable objetivo $Y$ la cual es cuantitativa y es de interes para el investigador.
Se quiere construir una función $f(X)$ donde $X=(X_1, \ldots, X_p)$ es un conjunto de variables exogenas que se utilizaran para pronosticar a $Y$.

En un modelo de regresión lineal, se usan las funciones del tipo:
$$Y=\beta_0 +\beta_1X_1+\beta_2X_2+...+\beta_pX_p +\epsilon $$ 
o de la forma más general
$$f_0(Y)=\beta_0 +\beta_1 f_1(X_1)+\beta_2 f_2(X_2)+...+\beta_p f_p(X_p) +\epsilon $$ 
donde $\epsilon$ se conoce como el error o ruido del modelo.

Sobre este error se realizan varios supuestos para que el modelo tenga validez estadística.
1. Normalidad o gaussianidad : Campana de Gauss
2. Homocedasticidad : La variabilidad de mi modelo no depende de las X
3. Independencia


## Datos
Los datos están [acá](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime+Unnormalized#) 


In [7]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Violencia.csv"
violencia=pd.read_csv(url, sep=";", decimal=",", na_values="?",index_col=0)
violencia

,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
communityname,,,,,,,,,,,,,,,,,,,,,
BerkeleyHeightstownship,NJ,39.0,5320.0,1,11980,3.10,1.37,91.78,6.50,1.88,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
Marpletownship,PA,45.0,47616.0,1,23123,2.82,0.80,95.57,3.44,0.85,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
Tigardcity,OR,NaN,NaN,1,29344,2.43,0.74,94.33,3.43,2.35,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
Gloversvillecity,NY,35.0,29443.0,1,16656,2.40,1.70,97.35,0.50,0.70,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
Bemidjicity,MN,7.0,5068.0,1,11245,2.76,0.53,89.16,1.17,0.52,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mercedcity,CA,NaN,NaN,10,56216,3.07,6.87,61.68,15.23,29.86,...,1376.0,2268.72,2563.0,4225.82,489.0,806.25,34.0,56.06,545.75,7356.84
Pinevillecity,LA,NaN,NaN,10,12251,2.68,21.18,76.65,1.52,1.29,...,104.0,860.43,574.0,4748.90,24.0,198.56,2.0,16.55,124.10,5824.44
Yucaipacity,CA,NaN,NaN,10,32824,2.46,0.52,92.62,0.98,11.00,...,628.0,1709.26,895.0,2435.97,179.0,487.19,8.0,21.77,353.83,4654.20


## Descriptiva de la variable objetivo

In [8]:
violencia["murdPerPop"].describe()

count    2215.000000
mean        5.859296
std         9.156829
min         0.000000
25%         0.000000
50%         2.170000
75%         8.365000
max        91.090000
Name: murdPerPop, dtype: float64

In [9]:
fig=px.histogram(x=violencia.murdPerPop)
fig.show()

In [10]:
### Para que el 0 no genere lío
fig=px.histogram(x=np.log(violencia.murdPerPop+1)) ## Porque el logaritmo de cero 
fig.show()

In [11]:
fig=px.histogram(violencia[violencia["murdPerPop"]>0], x="murdPerPop") ## Porque el logaritmo de cero 
fig.show()

In [12]:
### Solo dejaré los mayores a 0
con_asesinatos = violencia[violencia["murdPerPop"]>0]
con_asesinatos["log_murdPerPop"] = np.log(con_asesinatos["murdPerPop"])
fig=px.histogram(con_asesinatos, x="log_murdPerPop") ## Porque el logaritmo de cero 
fig.show()

<ipython-input-12-5f8d8e2622a9>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Variables para el analisis
householdsize: mean people per household (numeric - decimal)

-- racepctblack: percentage of population that is african american (numeric - decimal)

-- racePctWhite: percentage of population that is caucasian (numeric - decimal)

-- racePctAsian: percentage of population that is of asian heritage (numeric - decimal)

-- racePctHisp: percentage of population that is of hispanic heritage (numeric - decimal)

-- agePct12t21: percentage of population that is 12-21 in age (numeric - decimal)

-- agePct12t29: percentage of population that is 12-29 in age (numeric - decimal)

-- agePct16t24: percentage of population that is 16-24 in age (numeric - decimal)

-- agePct65up: percentage of population that is 65 and over in age (numeric - decimal)

-- numbUrban: number of people living in areas classified as urban (numeric - expected to be integer)

-- pctUrban: percentage of people living in areas classified as urban (numeric - decimal)

-- medIncome: median household income (numeric - may be integer)

In [13]:
base_interes=con_asesinatos[["murdPerPop","log_murdPerPop",'householdsize', 'racepctblack', 'racePctWhite', 'racePctAsian',
       'racePctHisp', 'agePct12t21', 'agePct12t29', 'agePct16t24',
       'agePct65up', 'pctUrban','medIncome']] ## Selecciono a Y y a las X, la de estado para estratificar
base_sinNA=base_interes.dropna()

In [14]:
base_interes.isnull().sum()

murdPerPop        0
log_murdPerPop    0
householdsize     0
racepctblack      0
racePctWhite      0
racePctAsian      0
racePctHisp       0
agePct12t21       0
agePct12t29       0
agePct16t24       0
agePct65up        0
pctUrban          0
medIncome         0
dtype: int64

## Discusión 
¿Qué haría con los NAs?

1. Reportar la cantidad y proporción
2. La primera opción es volverse sicario de datos.
3. Sí es cualitativa, una opción es crear una nueva categoría _sin información_
4. Sí es cuantitativa, usar un algoritmo como k-nn (K vecinos más cercanos), K-nearest neigrborhood.
5. Cuidado, incluir imputación de faltantes, empieza a volver sinteticos los datos.
6. No reemplazar por la media, o por la mediana. (reduce la información en los datos)

## Correlaciones

In [15]:
base_sinNA.corr()

,murdPerPop,log_murdPerPop,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,pctUrban,medIncome
murdPerPop,1.000000,0.863167,0.060675,0.621274,-0.603766,-0.097570,0.058725,0.066672,0.037649,0.018067,0.029685,0.003211,-0.355295
log_murdPerPop,0.863167,1.000000,0.051309,0.561699,-0.562104,-0.116318,0.097260,0.054197,0.012654,-0.000557,0.080517,-0.094397,-0.418555
householdsize,0.060675,0.051309,1.000000,-0.054849,-0.328699,0.220697,0.631510,0.485504,0.397482,0.291630,-0.573074,-0.013744,0.122278
racepctblack,0.621274,0.561699,-0.054849,1.000000,-0.793347,-0.175984,-0.201404,0.123450,0.073934,0.075168,0.056467,-0.029868,-0.392903
racePctWhite,-0.603766,-0.562104,-0.328699,-0.793347,1.000000,-0.224118,-0.315280,-0.199463,-0.202387,-0.130197,0.158142,-0.033996,0.300560
racePctAsian,-0.097570,-0.116318,0.220697,-0.175984,-0.224118,1.000000,0.191132,-0.017814,0.082546,0.036254,-0.246819,0.212352,0.384469
racePctHisp,0.058725,0.097260,0.631510,-0.201404,-0.315280,0.191132,1.000000,0.163269,0.190053,0.073316,-0.253370,0.056625,-0.064678
agePct12t21,0.066672,0.054197,0.485504,0.123450,-0.199463,-0.017814,0.163269,1.000000,0.853203,0.915342,-0.409461,-0.196181,-0.288583
agePct12t29,0.037649,0.012654,0.397482,0.073934,-0.202387,0.082546,0.190053,0.853203,1.000000,0.942607,-0.559362,-0.020933,-0.224343
agePct16t24,0.018067,-0.000557,0.291630,0.075168,-0.130197,0.036254,0.073316,0.915342,0.942607,1.000000,-0.347109,-0.085795,-0.257991


In [16]:
base_interes.corr()

,murdPerPop,log_murdPerPop,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,pctUrban,medIncome
murdPerPop,1.000000,0.863167,0.060675,0.621274,-0.603766,-0.097570,0.058725,0.066672,0.037649,0.018067,0.029685,0.003211,-0.355295
log_murdPerPop,0.863167,1.000000,0.051309,0.561699,-0.562104,-0.116318,0.097260,0.054197,0.012654,-0.000557,0.080517,-0.094397,-0.418555
householdsize,0.060675,0.051309,1.000000,-0.054849,-0.328699,0.220697,0.631510,0.485504,0.397482,0.291630,-0.573074,-0.013744,0.122278
racepctblack,0.621274,0.561699,-0.054849,1.000000,-0.793347,-0.175984,-0.201404,0.123450,0.073934,0.075168,0.056467,-0.029868,-0.392903
racePctWhite,-0.603766,-0.562104,-0.328699,-0.793347,1.000000,-0.224118,-0.315280,-0.199463,-0.202387,-0.130197,0.158142,-0.033996,0.300560
racePctAsian,-0.097570,-0.116318,0.220697,-0.175984,-0.224118,1.000000,0.191132,-0.017814,0.082546,0.036254,-0.246819,0.212352,0.384469
racePctHisp,0.058725,0.097260,0.631510,-0.201404,-0.315280,0.191132,1.000000,0.163269,0.190053,0.073316,-0.253370,0.056625,-0.064678
agePct12t21,0.066672,0.054197,0.485504,0.123450,-0.199463,-0.017814,0.163269,1.000000,0.853203,0.915342,-0.409461,-0.196181,-0.288583
agePct12t29,0.037649,0.012654,0.397482,0.073934,-0.202387,0.082546,0.190053,0.853203,1.000000,0.942607,-0.559362,-0.020933,-0.224343
agePct16t24,0.018067,-0.000557,0.291630,0.075168,-0.130197,0.036254,0.073316,0.915342,0.942607,1.000000,-0.347109,-0.085795,-0.257991


## Conjunto de entrenamiento y prueba


1. Vamos a ser religiosos, mientras el café, leemos este [blog](https://www.aprendemachinelearning.com/sets-de-entrenamiento-test-validacion-cruzada/)

## Vamos a hacer un modelo con murdPerPop

1. Solo una variable explicativa
2. La variable sin transformar

In [17]:
X=base_sinNA[["medIncome"]] ###
###  La matriz de las variables explicativas 
### las Y 
Y=base_sinNA["murdPerPop"] ### Selecciono la variable objetivo
X.shape

(1189, 1)

In [18]:
### X_train y Y_train tendrán los mismos individuos (un 80%)
### X_test y Y_test tendrán el 20% restante
### Esta división se hace aleatoria
### El random_state es para garantizar que a otra persona le de los mismo
### en la selección aleatoria
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, train_size=0.8, random_state=20) ## Muestreo aleatorio simple
X_test.shape

(238, 1)

In [19]:
#### Sólo lo hago con los datos de entrenamiento
#### las 1 variables y las 951 filas
RegresionLineal=linear_model.LinearRegression().fit(X_train, Y_train) ## Entrenar la regresión

## ¿Qué hizo el modelo?

In [20]:
### Los parametros estimados
print("Intercepto", RegresionLineal.intercept_)
print("betas", pd.DataFrame(RegresionLineal.coef_, index=X_train.columns))

Intercepto 20.30080047373381
betas                   0
medIncome -0.000314


In [21]:
## Para obtener un valor pronosticado,
## Simplemente le ingreso los valores de X
RegresionLineal.predict(X_test)[0:10]

array([10.14189959,  9.29216048,  9.38064882, 12.2135309 , 10.43278148,
       15.43456928, 10.58371372,  9.0715672 , 13.69241584,  6.74796377])

In [22]:
Y_pron = pd.DataFrame(RegresionLineal.predict(X_test), index=Y_test.index, columns=["Pronostico"])
Y_pron["Reales"] = Y_test
Y_pron["Media"] = np.mean(Y_test)
Y_pron

,Pronostico,Reales,Media
communityname,,,
Appletoncity,10.141900,2.85,11.258613
Mantecacity,9.292160,2.25,11.258613
Riversidecity,9.380649,13.59,11.258613
Rosenbergcity,12.213531,4.56,11.258613
Summervilletown,10.432781,4.43,11.258613
...,...,...,...
Norcocity,4.111199,4.04,11.258613
Schaumburgvillage,5.543644,1.40,11.258613
Joplincity,14.207029,11.56,11.258613


In [23]:
## Y ahora uno con el logaritmo
RegresionLineal_log=linear_model.LinearRegression().fit(X_train, np.log(Y_train)) ## Entrenar la regresión

In [24]:
print("Intercepto", RegresionLineal.intercept_)
print("betas", pd.DataFrame(RegresionLineal.coef_, index=X_train.columns))

Intercepto 20.30080047373381
betas                   0
medIncome -0.000314


In [25]:
Y_pron["Pron_log"]=np.exp(RegresionLineal_log.predict(X_test))
Y_pron

,Pronostico,Reales,Media,Pron_log
communityname,,,,
Appletoncity,10.141900,2.85,11.258613,7.244841
Mantecacity,9.292160,2.25,11.258613,6.656900
Riversidecity,9.380649,13.59,11.258613,6.715831
Rosenbergcity,12.213531,4.56,11.258613,8.905135
Summervilletown,10.432781,4.43,11.258613,7.457812
...,...,...,...,...
Norcocity,4.111199,4.04,11.258613,3.973392
Schaumburgvillage,5.543644,1.40,11.258613,4.582728
Joplincity,14.207029,11.56,11.258613,10.861065


In [26]:
### MSE (Para el modelo sencillo) 
### Mean Square Error

sum((Y_pron["Reales"]-Y_pron["Pronostico"])**2)/Y_pron.shape[0]


88.17394734232067

In [27]:
### RMSE (Para el modelo sencillo) 
### Root Mean Square Error

np.sqrt(sum((Y_pron["Reales"]-Y_pron["Pronostico"])**2)/Y_pron.shape[0])

9.39009836702048

In [28]:
### MSE (Para el modelo sencillo) 
### Mean Square Error

sum((Y_pron["Reales"]-Y_pron["Pron_log"])**2)/Y_pron.shape[0]

99.62410423962817

In [29]:
### RMSE (Para el modelo sencillo) 
### Root Mean Square Error

np.sqrt(sum((Y_pron["Reales"]-Y_pron["Pron_log"])**2)/Y_pron.shape[0])

9.981187516504646

## Tarea para el 20

1. Calcular el R2, el MAPE, el MSE y el RMSE e interpretarlos.

## ¿Y este modelo es bueno?
Observar su comportamiento en pronostico.

Entran a jugar las metricas [acá](https://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score)

## R cuadrado

In [30]:
from sklearn.metrics import r2_score
Y_pred_train=RegresionLineal.predict(X_train) ### Entrenamiento
Y_pred_test=RegresionLineal.predict(X_test) ## Prueba
print("R2 train",np.round(r2_score(Y_train, Y_pred_train),2)*100, "%")
print("R2 prueba",np.round(r2_score(Y_test, Y_pred_test),2)*100, "%")

R2 train 12.0 %
R2 prueba 15.0 %


### MSE

In [31]:
from sklearn.metrics import mean_squared_error
Y_pred_train=RegresionLineal.predict(X_train) ### Entrenamiento
Y_pred_test=RegresionLineal.predict(X_test) ## Prueba
print("MSE train",np.round(mean_squared_error(Y_train, Y_pred_train),2))
print("MSE prueba",np.round(mean_squared_error(Y_test, Y_pred_test),2))

MSE train 88.25
MSE prueba 88.17


## MAPE

In [32]:
from sklearn.metrics import mean_absolute_percentage_error
Y_pred_train=RegresionLineal.predict(X_train) ### Entrenamiento
Y_pred_test=RegresionLineal.predict(X_test) ## Prueba
print("MAPE train",np.round(mean_absolute_percentage_error(Y_train, Y_pred_train),2)*100, "%")
print("MAPE prueba",np.round(mean_absolute_percentage_error(Y_test, Y_pred_test),2)*100, "%")

MAPE train 105.0 %
MAPE prueba 96.0 %


## Y la parte estadística

Modelo con la variable sin transformar.
Vamos a usar la libreria [statsmodels](https://www.statsmodels.org/stable/index.html)

In [33]:
X_train1 = sm.add_constant(X_train)
### Lo pongo en miles
X_train1["medIncome"] = X_train1["medIncome"]/1000 
ModeloLineal = sm.OLS( Y_train,X_train1)
Resultado = ModeloLineal.fit()

In [34]:
print(Resultado.summary())

                            OLS Regression Results                            
Dep. Variable:             murdPerPop   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     127.0
Date:                Wed, 26 Apr 2023   Prob (F-statistic):           9.75e-28
Time:                        00:11:54   Log-Likelihood:                -3479.7
No. Observations:                 951   AIC:                             6963.
Df Residuals:                     949   BIC:                             6973.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.3008      0.894     22.707      0.0

In [35]:
X_train[0:6]

,medIncome
communityname,
Miamicity,16925
ScotchPlainstownship,58194
Hendersoncity,22085
FortSmithcity,23835
Sylacaugacity,19950
Spokanecity,22192


In [36]:
Y_train[0:6]

communityname
Miamicity               33.53
ScotchPlainstownship     4.61
Hendersoncity            3.75
FortSmithcity            5.30
Sylacaugacity            7.17
Spokanecity              6.63
Name: murdPerPop, dtype: float64

## Y con la variable transformada

In [37]:
X_train1 = sm.add_constant(X_train)
ModeloLineal = sm.OLS( np.log(Y_train),X_train1)
Resultado = ModeloLineal.fit()

In [38]:
print(Resultado.summary())

                            OLS Regression Results                            
Dep. Variable:             murdPerPop   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     197.8
Date:                Wed, 26 Apr 2023   Prob (F-statistic):           5.96e-41
Time:                        00:11:55   Log-Likelihood:                -1075.5
No. Observations:                 951   AIC:                             2155.
Df Residuals:                     949   BIC:                             2165.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9921      0.071     41.935      0.0

## Discusión de los resultados


## Ahora una con todos los datos

In [39]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Violencia.csv"
violencia=pd.read_csv(url, sep=";", decimal=",", na_values="?",index_col=0)
base_interes=violencia[["murdPerPop",'householdsize', 'racepctblack', 'racePctWhite', 'racePctAsian',
       'racePctHisp', 'agePct12t21', 'agePct12t29', 'agePct16t24',
       'agePct65up', 'pctUrban','medIncome']] ## Selecciono a Y y a las X, la de estado para estratificar
base_sinNA=base_interes.dropna()

In [40]:
X=base_sinNA.drop("murdPerPop", axis=1)###
###  La matriz de las variables explicativas 
### las Y 
Y=base_sinNA["murdPerPop"] ### Selecciono la variable objetivo
X.shape

(2215, 11)

In [41]:
### X_train y Y_train tendrán los mismos individuos (un 80%)
### X_test y Y_test tendrán el 20% restante
### Esta división se hace aleatoria
### El random_state es para garantizar que a otra persona le de los mismo
### en la selección aleatoria
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, train_size=0.8, random_state=20) ## Muestreo aleatorio simple
X_test.shape

(443, 11)

In [42]:
#### Sólo lo hago con los datos de entrenamiento
#### las 1 variables y las 951 filas
RegresionLineal=linear_model.LinearRegression().fit(X_train, Y_train) ## Entrenar la regresión

In [43]:
### Los parametros estimados
print("Intercepto", RegresionLineal.intercept_)
print("betas", pd.DataFrame(RegresionLineal.coef_, index=X_train.columns))

Intercepto 47.41729622038709
betas                       0
householdsize -0.620412
racepctblack   0.123773
racePctWhite  -0.274934
racePctAsian  -0.154905
racePctHisp    0.013229
agePct12t21   -0.257808
agePct12t29   -0.581248
agePct16t24    0.589745
agePct65up    -0.223392
pctUrban       0.016372
medIncome     -0.000134


In [44]:
px.scatter(X_train, x="racePctWhite", y="racepctblack")

## Una importancia de variables

In [45]:
X_train1 = sm.add_constant(X_train)
ModeloLineal = sm.OLS(Y_train,X_train1)
Resultado = ModeloLineal.fit()

In [46]:
print(Resultado.summary())

                            OLS Regression Results                            
Dep. Variable:             murdPerPop   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.508
Method:                 Least Squares   F-statistic:                     167.5
Date:                Wed, 26 Apr 2023   Prob (F-statistic):          3.04e-264
Time:                        00:11:55   Log-Likelihood:                -5779.5
No. Observations:                1772   AIC:                         1.158e+04
Df Residuals:                    1760   BIC:                         1.165e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            47.4173      6.338      7.481

## R cuadrado

In [47]:
from sklearn.metrics import r2_score
Y_pred_train=RegresionLineal.predict(X_train) ### Entrenamiento
Y_pred_test=RegresionLineal.predict(X_test) ## Prueba
print("R2 train",np.round(r2_score(Y_train, Y_pred_train),2)*100, "%")
print("R2 prueba",np.round(r2_score(Y_test, Y_pred_test),2)*100, "%")

R2 train 51.0 %
R2 prueba 49.0 %


# MSE

In [48]:
from sklearn.metrics import mean_squared_error
Y_pred_train=RegresionLineal.predict(X_train) ### Entrenamiento
Y_pred_test=RegresionLineal.predict(X_test) ## Prueba
print("MSE train",np.round(mean_squared_error(Y_train, Y_pred_train),2))
print("MSE prueba",np.round(mean_squared_error(Y_test, Y_pred_test),2))

MSE train 39.86
MSE prueba 47.11


## MAPE

In [49]:
from sklearn.metrics import mean_absolute_percentage_error
Y_pred_train=RegresionLineal.predict(X_train) ### Entrenamiento
Y_pred_test=RegresionLineal.predict(X_test) ## Prueba
print("MAPE train",np.round(mean_absolute_percentage_error(Y_train, Y_pred_train),2)*100, "%")
print("MAPE prueba",np.round(mean_absolute_percentage_error(Y_test, Y_pred_test),2)*100, "%")

MAPE train 5.925507956706963e+17 %
MAPE prueba 6.674388419044102e+17 %


In [50]:
Y_train

communityname
CottageGrovecity            0.00
Daviscity                   2.06
Tauntoncity                 0.00
HarperWoodscity             0.00
Lubbockcity                 8.58
                           ...  
Ramseycity                 12.62
Gilberttown                 0.00
WarrensvilleHeightscity    18.98
Eulesscity                  7.37
Mariettacity                6.57
Name: murdPerPop, Length: 1772, dtype: float64

In [51]:
Y_pred_train

array([ 0.37715018,  4.70179167,  2.36245663, ..., 38.30362725,
        4.3074934 ,  1.99238364])

### ¿Qué pasa con el MAPE?

### Ejercicio

1. Sumarle a todos los asesinatos un valor de 2
2. Ajustar el modelo con la variable murdPerPop
3. ¿Qué pasa con el MAPE?

## Vamos con variables dummies

1. Vamos a leer este [proyecto](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=train.csv)

2. Descripción de los [datos](https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Casas/data_description.txt)

In [52]:
url ="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Casas/train.csv"
train = pd.read_csv(url, index_col=0)
url ="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Casas/test.csv"
test = pd.read_csv(url)

In [53]:
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [54]:
train[["Alley", "Fence"]].dtypes

Alley    object
Fence    object
dtype: object

In [55]:
pd.get_dummies(train[["Alley", "Fence"]], drop_first=True)

,Alley_Pave,Fence_GdWo,Fence_MnPrv,Fence_MnWw
Id,,,,
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
...,...,...,...,...
1456,0,0,0,0
1457,0,0,1,0
1458,0,0,0,0


In [56]:
train.groupby("Alley").size()

Alley
Grvl    50
Pave    41
dtype: int64

In [57]:
train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

## Primer paso
1. Seleccionar a X
2. Seleccionar a Y

In [58]:
X = train[["MSZoning", "LotFrontage", "LotArea", "Street", "YearBuilt", "MasVnrArea"]]
### Las dummies se deben hacer antes de entrenamiento y prueba
X = pd.get_dummies(X, drop_first=True)
Y = train["SalePrice"]

In [59]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, train_size=0.8, random_state=20) ## Muestreo aleatorio simple
X_test.shape

(292, 9)

In [60]:
X_train

,LotFrontage,LotArea,YearBuilt,MasVnrArea,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave
Id,,,,,,,,,
1138,54.0,6342,1875,0.0,0,0,1,0,1
1336,80.0,9650,1977,360.0,0,0,1,0,1
460,NaN,7015,1950,161.0,0,0,1,0,1
116,34.0,3230,1999,1129.0,1,0,0,0,1
909,NaN,8885,1983,0.0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
925,79.0,10240,1980,157.0,0,0,1,0,1
1248,NaN,12328,1976,335.0,0,0,1,0,1
272,73.0,39104,1954,0.0,0,0,1,0,1


## Variables dummy



In [61]:
X_train

,LotFrontage,LotArea,YearBuilt,MasVnrArea,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave
Id,,,,,,,,,
1138,54.0,6342,1875,0.0,0,0,1,0,1
1336,80.0,9650,1977,360.0,0,0,1,0,1
460,NaN,7015,1950,161.0,0,0,1,0,1
116,34.0,3230,1999,1129.0,1,0,0,0,1
909,NaN,8885,1983,0.0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
925,79.0,10240,1980,157.0,0,0,1,0,1
1248,NaN,12328,1976,335.0,0,0,1,0,1
272,73.0,39104,1954,0.0,0,0,1,0,1


## Datos faltantes

In [62]:
X_train.isnull().sum()

LotFrontage    208
LotArea          0
YearBuilt        0
MasVnrArea       7
MSZoning_FV      0
MSZoning_RH      0
MSZoning_RL      0
MSZoning_RM      0
Street_Pave      0
dtype: int64

1. Eliminar la variable, pierdo 1168-208 = 950
2. Eliminar los 208 datos, pierdo 208*9 = 1800 datos
3. Imputar los datos = Agrego 208 datos artificiales

## KNN = k-nearest_neigbors

Busca los k vecinos más cercanos en todas las variables con distancia euclidiana, [acá](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html).

In [63]:
### Imputar
## k vecinos más cercanos
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
X_train_imp = imputer.fit_transform(X_train)

In [64]:
X_train_imp

array([[5.4000e+01, 6.3420e+03, 1.8750e+03, ..., 1.0000e+00, 0.0000e+00,
        1.0000e+00],
       [8.0000e+01, 9.6500e+03, 1.9770e+03, ..., 1.0000e+00, 0.0000e+00,
        1.0000e+00],
       [8.1500e+01, 7.0150e+03, 1.9500e+03, ..., 1.0000e+00, 0.0000e+00,
        1.0000e+00],
       ...,
       [7.3000e+01, 3.9104e+04, 1.9540e+03, ..., 1.0000e+00, 0.0000e+00,
        1.0000e+00],
       [4.1000e+01, 5.3300e+03, 2.0000e+03, ..., 1.0000e+00, 0.0000e+00,
        1.0000e+00],
       [7.3000e+01, 9.7350e+03, 2.0060e+03, ..., 1.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [65]:
X_train_imp = pd.DataFrame(X_train_imp, columns=X_train.columns, index=X_train.index)
X_train_imp

,LotFrontage,LotArea,YearBuilt,MasVnrArea,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave
Id,,,,,,,,,
1138,54.0,6342.0,1875.0,0.0,0.0,0.0,1.0,0.0,1.0
1336,80.0,9650.0,1977.0,360.0,0.0,0.0,1.0,0.0,1.0
460,81.5,7015.0,1950.0,161.0,0.0,0.0,1.0,0.0,1.0
116,34.0,3230.0,1999.0,1129.0,1.0,0.0,0.0,0.0,1.0
909,73.5,8885.0,1983.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
925,79.0,10240.0,1980.0,157.0,0.0,0.0,1.0,0.0,1.0
1248,94.5,12328.0,1976.0,335.0,0.0,0.0,1.0,0.0,1.0
272,73.0,39104.0,1954.0,0.0,0.0,0.0,1.0,0.0,1.0


## Entrenamiento del modelo

1. Todas las variables son importantes para la regresión, es decir, va a _usarlas_ todas.

2. Esto puede ser un problema si se tiene muchas variables

3. En esos casos se usa regresion lasso o regresion Ridge

In [69]:
modelo1 = linear_model.LinearRegression().fit(X_train_imp, Y_train)

In [67]:
linear_model.LinearRegression().fit(X_train, Y_train)

ValueError: ignored

## R cuadrado

In [70]:
from sklearn.metrics import r2_score
Y_pred_train=modelo1.predict(X_train_imp) ### Entrenamiento
Y_pred_test=modelo1.predict(X_test) ## Prueba
print("R2 train",np.round(r2_score(Y_train, Y_pred_train),2)*100, "%")
print("R2 prueba",np.round(r2_score(Y_test, Y_pred_test),2)*100, "%")

ValueError: ignored

In [ ]:
imputer = KNNImputer(n_neighbors=2)
X_test_imp = imputer.fit_transform(X_test)
X_test_imp = pd.DataFrame(X_test_imp, columns=X_test.columns, index=X_test.index)

In [ ]:
from sklearn.metrics import r2_score
Y_pred_train=modelo1.predict(X_train_imp) ### Entrenamiento
Y_pred_test=modelo1.predict(X_test_imp) ## Prueba
print("R2 train",np.round(r2_score(Y_train, Y_pred_train),2)*100, "%")
print("R2 prueba",np.round(r2_score(Y_test, Y_pred_test),2)*100, "%")

## MAPE

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
Y_pred_train=modelo1.predict(X_train_imp) ### Entrenamiento
Y_pred_test=modelo1.predict(X_test_imp) ## Prueba
print("MAPE train",np.round(mean_absolute_percentage_error(Y_train, Y_pred_train),2)*100, "%")
print("MAPE prueba",np.round(mean_absolute_percentage_error(Y_test, Y_pred_test),2)*100, "%")

# Modelo con todas las variables

In [71]:
perdidos = train.isnull().sum().reset_index(name="Conteo").sort_values("Conteo", ascending=False)
perdidos["Prop"] =perdidos["Conteo"]/train.shape[0]
perdidos[perdidos["Prop"]>0.2]["index"].to_list()

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']

In [72]:
X = train.drop(["SalePrice",'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'], axis=1)
### Las dummies se deben hacer antes de entrenamiento y prueba
X = pd.get_dummies(X, drop_first=True)
Y = train["SalePrice"]

In [73]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, train_size=0.8, random_state=20) ## Muestreo aleatorio simple
X_test.shape

(292, 232)

In [74]:
X_train.shape

(1168, 232)

In [75]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1138,50,54.0,6342,5,8,1875,1996,0.0,0,0,...,0,0,0,0,1,0,0,0,1,0
1336,20,80.0,9650,6,5,1977,1977,360.0,686,0,...,0,0,0,0,1,0,0,0,1,0
460,50,NaN,7015,5,4,1950,1950,161.0,185,0,...,0,0,0,0,1,0,0,0,1,0
116,160,34.0,3230,6,5,1999,1999,1129.0,419,0,...,0,0,0,0,1,0,0,0,1,0
909,20,NaN,8885,5,5,1983,1983,0.0,301,324,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,20,79.0,10240,6,6,1980,1980,157.0,625,1061,...,0,0,0,0,1,0,0,0,1,0
1248,80,NaN,12328,6,5,1976,1976,335.0,539,0,...,0,0,0,0,1,0,0,0,1,0
272,20,73.0,39104,7,7,1954,2005,0.0,226,1063,...,0,0,0,0,1,0,0,0,1,0


In [76]:
X_train_imp = imputer.fit_transform(X_train)
X_train_imp = pd.DataFrame(X_train_imp, columns=X_train.columns, index=X_train.index)
modelo1 = linear_model.LinearRegression().fit(X_train_imp, Y_train)

In [77]:
modelo1.coef_

array([-3.52151050e+01,  3.30217307e+00,  7.29041706e-01,  6.20100245e+03,
        5.45414685e+03,  2.80024185e+02,  7.13906493e+01,  3.60369874e+01,
        1.24316342e+01,  9.94596469e+00, -3.60613970e+00,  1.87715265e+01,
        1.05084839e+01,  3.17460069e+01, -1.24223684e+01,  2.98319254e+01,
        1.32498101e+03,  3.37309044e+02,  4.82589293e+03,  2.62353600e+03,
       -3.49488572e+03, -1.26497645e+04,  1.57172552e+03,  3.46395710e+03,
       -2.26928141e+01,  3.68299245e+03,  1.44358209e+01,  1.88040797e+01,
        1.32479833e+01,  1.00683784e+01,  4.06812127e+01,  1.67393480e+01,
        3.23071202e+01, -6.55442244e-02, -5.28928132e+02, -4.12917694e+02,
        3.15047793e+04,  1.81509227e+04,  2.00339209e+04,  1.86468578e+04,
        2.70187208e+04,  5.83373897e+03,  7.16339762e+03,  1.73251340e+03,
        1.18750615e+04, -7.53921761e+03,  8.67781052e+03, -4.22907229e+04,
        1.16546291e+04, -9.54782916e+03, -1.56557423e+04, -1.42722999e+03,
        9.66273351e+03, -

In [78]:
imputer = KNNImputer(n_neighbors=2)
X_test_imp = imputer.fit_transform(X_test)
X_test_imp = pd.DataFrame(X_test_imp, columns=X_test.columns, index=X_test.index)

## Sobreajuste

1. Si la diferencia entre entrenamiento y prueba del $R^2$ supera el 10%, se habla de sobreajuste y deben quitarse parametros o variables en el entrenamiento.

In [79]:
from sklearn.metrics import r2_score
Y_pred_train=modelo1.predict(X_train_imp) ### Entrenamiento
Y_pred_test=modelo1.predict(X_test_imp) ## Prueba
print("R2 train",np.round(r2_score(Y_train, Y_pred_train),2)*100, "%")
print("R2 prueba",np.round(r2_score(Y_test, Y_pred_test),2)*100, "%")

R2 train 93.0 %
R2 prueba 68.0 %


In [80]:

print("MAPE train",np.round(mean_absolute_percentage_error(Y_train, Y_pred_train),2)*100, "%")
print("MAPE prueba",np.round(mean_absolute_percentage_error(Y_test, Y_pred_test),2)*100, "%")

MAPE train 8.0 %
MAPE prueba 11.0 %


## random feature selection

1. La documentación está [acá](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)


In [82]:
## Use validacion cruzada  (Se me salió de los cojones)
## Lo renuncian
regresion = linear_model.LinearRegression()
selector = RFE(regresion, n_features_to_select=20, step=1)
selector = selector.fit(X_train_imp, Y_train)
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True,  True,  True,  True, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,

In [83]:
X_train_imp.columns[selector.support_]

Index(['Neighborhood_Crawfor', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_StoneBr', 'Condition2_PosN', 'RoofMatl_CompShg',
       'RoofMatl_Membran', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv',
       'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'Exterior1st_ImStucc',
       'ExterQual_Fa', 'ExterQual_Gd', 'ExterQual_TA', 'BsmtExposure_Gd',
       'HeatingQC_Po', 'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA'],
      dtype='object')

In [84]:
X_train_imp

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1138,50.0,54.0,6342.0,5.0,8.0,1875.0,1996.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1336,20.0,80.0,9650.0,6.0,5.0,1977.0,1977.0,360.0,686.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
460,50.0,58.5,7015.0,5.0,4.0,1950.0,1950.0,161.0,185.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
116,160.0,34.0,3230.0,6.0,5.0,1999.0,1999.0,1129.0,419.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
909,20.0,70.0,8885.0,5.0,5.0,1983.0,1983.0,0.0,301.0,324.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,20.0,79.0,10240.0,6.0,6.0,1980.0,1980.0,157.0,625.0,1061.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1248,80.0,74.0,12328.0,6.0,5.0,1976.0,1976.0,335.0,539.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
272,20.0,73.0,39104.0,7.0,7.0,1954.0,2005.0,0.0,226.0,1063.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


## Cuantas variables entonces?
Validación cruzada




## Tarea con los datos de casas

1. Ya tenemos los datos dividios en train y test, además de tener imputados los faltantes.

2. El conjunto de datos de entrenamiento lo vamos a dividir en 
- El 80% para entrenamiento
- El 20% para validación

3. En el conjunto de datos de entrenamiento, entrenar 5 modelos cada uno con 5,10,15,20,25 variables.

4. Cada modelo probarlo en el conjunto de validación, con el $R^2$ y seleccionan el más alto. (por ejemplo el de 25 variables)

5. Ese modelo es el que van a usar para el conjunto de datos de prueba. 

In [91]:
X_train_80, X_valid_20, Y_train_80, Y_valid_20 = train_test_split(X_train_imp, Y_train, test_size=0.2)

In [98]:
## Modelo 1, con 1 variable
regresion = linear_model.LinearRegression()
selector = RFE(regresion, n_features_to_select=80, step=1)

selector = selector.fit(X_train_80, Y_train_80)
X_train_imp.columns[selector.support_]

Index(['OverallQual', 'FullBath', 'KitchenAbvGr', 'Fireplaces', 'GarageCars',
       'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
       'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR3',
       'Neighborhood_BrkSide', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_SawyerW',
       'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Veenker',
       'Condition1_PosA', 'Condition1_RRAe', 'Condition1_RRNe',
       'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosN',
       'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'RoofStyle_Gable', 'RoofStyle_Gambrel',
       'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed',
       'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Roll',
       'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl',
       'Exterior1st_AsphShn', 'Exterior1st_ImStucc', 'Exterior1st_Wd Sdng',
       'Exterior1st_WdShi

In [142]:
xSelec = ['OverallQual', 'FullBath', 'KitchenAbvGr', 'Fireplaces', 'GarageCars',
       'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
       'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR3',
       'Neighborhood_BrkSide', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_SawyerW',
       'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Veenker',
       'Condition1_PosA', 'Condition1_RRAe', 'Condition1_RRNe',
       'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosN',
       'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'RoofStyle_Gable', 'RoofStyle_Gambrel',
       'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed',
       'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Roll',
       'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl',
       'Exterior1st_AsphShn', 'Exterior1st_ImStucc', 'Exterior1st_Wd Sdng',
       'Exterior1st_WdShing', 'Exterior2nd_Other', 'Exterior2nd_Wd Sdng',
       'ExterQual_Fa', 'ExterQual_Gd', 'ExterQual_TA', 'ExterCond_Gd',
       'ExterCond_TA', 'Foundation_Slab', 'BsmtQual_Fa', 'BsmtQual_Gd',
       'BsmtQual_TA', 'BsmtCond_Gd', 'BsmtCond_Po', 'BsmtCond_TA',
       'BsmtExposure_Gd', 'BsmtFinType1_GLQ', 'BsmtFinType1_Unf',
       'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'Functional_Maj2',
       'Functional_Sev', 'GarageType_CarPort', 'GarageQual_Fa',
       'GarageQual_Po', 'GarageQual_TA', 'SaleType_CWD', 'SaleType_New',
       'SaleCondition_Alloca', 'SaleCondition_Partial']
modelo1 = linear_model.LinearRegression().fit(X_train_80[xSelec], Y_train_80)
Y_pred_train=modelo1.predict(X_train_80[xSelec]) ### Entrenamiento
Y_pred_test=modelo1.predict(X_valid_20[xSelec]) ## Prueba
print("R2 train",np.round(r2_score(Y_train_80, Y_pred_train),2)*100, "%")
print("R2 prueba",np.round(r2_score(Y_valid_20, Y_pred_test),2)*100, "%")

R2 train 87.0 %
R2 prueba 86.0 %


In [101]:
datos = X_valid_20
datos["Y"] = Y_valid_20
datos["Pron"] = Y_pred_test
datos

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Y,Pron
Id,,,,,,,,,,,,,,,,,,,,,
200,20.0,76.0,9591.0,8.0,5.0,2004.0,2005.0,262.0,1088.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,274900,344800.0
253,60.0,65.0,8366.0,6.0,5.0,2004.0,2004.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,173000,179872.0
1209,20.0,70.0,7763.0,5.0,7.0,1962.0,1980.0,0.0,504.0,108.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,140000,133824.0
1383,70.0,60.0,7200.0,7.0,7.0,1920.0,1950.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,157000,148864.0
989,60.0,85.0,12046.0,6.0,6.0,1976.0,1976.0,298.0,156.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,195000,191200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,30.0,60.0,7200.0,5.0,7.0,1940.0,1992.0,294.0,510.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,119000,156672.0
32,20.0,77.0,8544.0,5.0,6.0,1966.0,2006.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,149350,135776.0
837,30.0,90.0,8100.0,5.0,6.0,1948.0,1973.0,0.0,338.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,153500,128032.0


In [143]:
fig = px.scatter(datos[datos["Pron"]<1000000], x="Y", y="Pron")
fig.show()

In [127]:
datos[datos["Pron"]>1000000][xSelec[60:80]].describe()

,BsmtExposure_Gd,BsmtFinType1_GLQ,BsmtFinType1_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Sev,GarageType_CarPort,GarageQual_Fa,GarageQual_Po,GarageQual_TA,SaleType_CWD,SaleType_New,SaleCondition_Alloca,SaleCondition_Partial
count,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.000000,2.0,2.0,2.0,2.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.750000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0


In [136]:
X_train[["Heating_GasW","Heating_GasA", "Heating_Wall","Heating_Grav"]].describe()

,Heating_GasW,Heating_GasA,Heating_Wall,Heating_Grav
count,1168.000000,1168.000000,1168.000000,1168.000000
mean,0.014555,0.978596,0.002568,0.002568
std,0.119813,0.144789,0.050637,0.050637
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [141]:
train.groupby("Heating").size()

Heating
Floor       1
GasA     1428
GasW       18
Grav        7
OthW        2
Wall        4
dtype: int64

In [133]:
betas = pd.DataFrame(modelo1.coef_, index=xSelec, columns = ["Variable"])
betas.sort_values("Variable")

,Variable
Heating_GasA,-2.640766e+17
Heating_Wall,-2.640766e+17
Heating_Grav,-2.640766e+17
Heating_GasW,-2.640766e+17
Condition2_PosN,-1.149831e+05
...,...
RoofMatl_Membran,2.184868e+05
RoofMatl_Roll,2.532493e+05
RoofMatl_CompShg,2.607903e+05
RoofMatl_WdShake,3.023122e+05
